# Descriptive statistics

The following code describes the dataset used in the sentiment analysis

In [15]:
import pandas as pd
import pysentiment2 as ps
from google.colab import drive 
import numpy as np
pd.set_option('display.expand_frame_repr', False)

drive.mount("/content/gdrive")


data = pd.read_csv("gdrive/My Drive/Thesis/processed data/data_processed_withoutPS.csv")

#Prepare data for descriptive statistics
data["Date"] = pd.to_datetime(data["Date"], format = "%Y-%m-%d")
data.sort_values(by= "Date", inplace = True)
data.drop_duplicates(inplace = True)

#Function to split initial dataframe into dataframes grouped by year
def split_years(dt):
    dt["Year"] = dt["Date"].dt.year
    return [dt[dt["Year"] == y] for y in dt["Year"].unique()]

data_splt_years = split_years(data)
data_fill = []

for df_year_splt in data_splt_years:
    year = df_year_splt["Date"].iloc[0].year
    obs_count = len(df_year_splt)
    earliest_obs = df_year_splt["Date"].iloc[0]
    latest_obs = df_year_splt["Date"].iloc[-1]
    mean_word_count = df_year_splt["Word_count"].mean()
    company_count = df_year_splt["Ticker"].nunique()

    data_fill.append([year, obs_count, earliest_obs, latest_obs, mean_word_count, company_count])

#Calculate the metrics for the whole dataset
obs_count = len(data)
earliest_obs = data["Date"].iloc[0]
latest_obs = data["Date"].iloc[-1]
mean_word_count = data["Word_count"].mean()
company_count = data["Ticker"].nunique()

data_fill.append(["All years", obs_count, earliest_obs, latest_obs, mean_word_count, company_count])

df_by_year = pd.DataFrame(data_fill ,columns = ["Year", "Observations", "Earliest Observation", "Latest Observation", "Mean Word Count", "Company Count"])

print(df_by_year)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
         Year  Observations Earliest Observation Latest Observation  Mean Word Count  Company Count
0        2009           112           2009-06-26         2009-12-31       783.508929             71
1        2010          2836           2010-01-01         2010-12-31       408.525035            893
2        2011          4331           2011-01-01         2011-12-30       428.202263           1039
3        2012         10462           2012-01-03         2012-12-31       311.080960           2379
4        2013         13480           2013-01-01         2013-12-31       258.271588           3210
5        2014         26715           2014-01-01         2014-12-31       274.373573           4188
6        2015         36496           2015-01-01         2015-12-31       317.943062           4973
7        2016         37777           2016-01-01         2016-12-31  

Get an overview of the data with respect to the companies

In [16]:
unique_data_company =  data.groupby("Ticker").nunique().sort_values(by = "Text", ascending = False)
df_unique_data_company = pd.DataFrame(unique_data_company, columns = ["Date", "Text", "Word_count", "Year"])

del df_unique_data_company["Word_count"]
df_unique_data_company.rename(columns={"Text":"News Articles"}, inplace=True)
df_unique_data_company["Average Articles p.d."] = round(df_unique_data_company["News Articles"] / df_unique_data_company["Date"], 2)

print(df_unique_data_company.head(20))

        Date  News Articles  Year  Average Articles p.d.
Ticker                                                  
AAPL    1826           7025    10                   3.85
AMZN    1009           2857    11                   2.83
FB      1008           2156     8                   2.14
TSLA    1114           2129     9                   1.91
SPX     1161           1909     9                   1.64
BA       708           1516    10                   2.14
NFLX     726           1197    10                   1.65
DIS      828           1177    10                   1.42
INTC     803           1114    11                   1.39
BAC      741           1101    11                   1.49
F        681           1015    10                   1.49
EFX      352           1010     8                   2.87
GLD      643            982    11                   1.53
GE       668            943    10                   1.41
DAX      707            934     6                   1.32
MSFT     666            883    

In the next step, we check the sentiment indices of the dataset given by the dictionary methods using the Harvard Psychsosocological Dictionary and the Loughran and McDonald Dictionary.

In [17]:
hiv4 = ps.HIV4()
lm = ps.LM()

data_fill = []

for df_year_splt in data_splt_years:

    year = df_year_splt["Date"].iloc[0].year

    hiv4_pos = []
    hiv4_neg = []
    hiv4_tone = []

    lm_pos = []
    lm_neg = []
    lm_tone = []

    for index, row in df_year_splt.iterrows():
      
        tokens_hiv4 = hiv4.tokenize(row["Text"])
        tokens_lm = lm.tokenize(row["Text"])

        score_hiv4 = hiv4.get_score(tokens_hiv4)
        score_lm = lm.get_score(tokens_lm)

        hiv4_pos.append(score_hiv4["Positive"])
        hiv4_neg.append(score_hiv4["Negative"])
        hiv4_tone.append(score_hiv4["Positive"] - score_hiv4["Negative"])

        lm_pos.append(score_lm["Positive"])
        lm_neg.append(score_lm["Negative"])
        lm_tone.append(score_lm["Positive"] - score_lm["Negative"])

    data_fill.append([str(year), sum(hiv4_pos), sum(hiv4_neg), sum(hiv4_tone), sum(lm_pos), sum(lm_neg), sum(lm_tone)])

df_by_year_dm = pd.DataFrame(data_fill ,columns = ["Year", "HIV4 positive", "HIV4 negative", "HIV4 tone", "LM_pos", "LM_neg", "LM_tone"])

df_by_year_dm.loc["Total"] = df_by_year_dm.sum(numeric_only=True, axis = 0)
df_by_year_dm.at["Total", "Year"] = "All years"

print(df_by_year_dm)

            Year  HIV4 positive  HIV4 negative  HIV4 tone     LM_pos     LM_neg   LM_tone
0           2009         6181.0         3358.0     2823.0      942.0     1804.0    -862.0
1           2010        75075.0        35830.0    39245.0    11625.0    16146.0   -4521.0
2           2011       123012.0        56017.0    66995.0    18170.0    24594.0   -6424.0
3           2012       222105.0        96681.0   125424.0    32302.0    37126.0   -4824.0
4           2013       212709.0       103582.0   109127.0    31041.0    39428.0   -8387.0
5           2014       468392.0       231155.0   237237.0    66679.0   108840.0  -42161.0
6           2015       763503.0       372421.0   391082.0   116756.0   164112.0  -47356.0
7           2016       968957.0       482229.0   486728.0   153482.0   182837.0  -29355.0
8           2017      1508858.0       637895.0   870963.0   264627.0   255755.0    8872.0
9           2018      4063978.0      1734882.0  2329096.0   740172.0   710158.0   30014.0
10        